In [1]:
%reload_ext watermark
%watermark -v -p tqdm,re,joblib,shutil,pandas,numpy,cv2,tensorflow,numpy,matplotlib,sklearn

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.34.0

tqdm      : 4.63.1
re        : 2.2.1
joblib    : 1.1.0
shutil    : unknown
pandas    : 1.4.1
numpy     : 1.22.3
cv2       : 4.6.0
tensorflow: 2.8.0
matplotlib: 3.5.1
sklearn   : 1.0.2



In [2]:
import pandas as pd
import numpy as np
import cv2,datetime
import tensorflow as tf

In [3]:
train=pd.read_csv("안진방향판별_train.tsv",sep='\t')
valid=pd.read_csv("안진방향판별_valid.tsv",sep='\t')

In [4]:
def my_generator(df):
    # 모두 같은 프레임은 제외하고 불러오게 설정
    def avi_to_img(file, video_len=32):
        data=list()
        cap = cv2.VideoCapture(file)
        cnt=0
        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret:
                if (cnt!=0):
                    before=data[-1][0]
                else:
                    before=np.zeros((240,320))

                frame=cv2.resize(frame,(320,240))
                frame=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
                frame=np.expand_dims(frame/255, axis=0)
                if (before!=frame).any():
                    data.append(frame)
                    cnt+=1
                cv2.waitKey(1)
            else:
                break
            if cnt==video_len:
                break
        cap.release()
        cv2.destroyAllWindows()
        if len(data)!=0:
            out = np.concatenate(data).astype('float')
            # # 이미지별 차이를 리턴
            out=np.diff(out,axis=0)[:].sum(axis=0)
            # # min max scaling 
            out=(((out-out.min())/(out.max()-out.min()))*255
                ).astype('u1')
            out=cv2.cvtColor(out,cv2.COLOR_GRAY2RGB)
            return out.copy()
        else:
            return np.zeros((240,320,3))

    def f():
        for label,video_file in df.sample(frac=1,random_state=42).values:
            video = avi_to_img(video_file)
            y = np.zeros((3))
            if label=="left":
                y[0]=1
            elif label=="right":
                y[2]=1
            else:
                y[1]=1
            yield video, y
    return f

In [5]:
tr_dataset=tf.data.Dataset.from_generator(
    my_generator(train[:].reset_index(drop=True)),
    output_shapes=((240,320,3), 3),
    output_types=('float32','float32'))
tr_gen =tr_dataset.batch(640).prefetch(tf.data.experimental.AUTOTUNE)

val_dataset=tf.data.Dataset.from_generator(
    my_generator(valid),
    output_shapes=((240,320,3), 3),
    output_types=('float32','float32'))
val_gen =val_dataset.batch(640).prefetch(tf.data.experimental.AUTOTUNE)

2022-12-17 20:48:56.719759: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-17 20:48:56.723266: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-17 20:48:56.723389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-17 20:48:56.723930: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
def my_cnn_model():
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    cnn_model = tf.keras.Sequential()
    cnn_model.add(tf.keras.layers.Input((240,320,3)))
    cnn_model.add(tf.keras.layers.Rescaling(1./255))
    # cnn_model.add(tf.keras.layers.Resizing(256,320,name='resize'))

    cnn_model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
    cnn_model.add(tf.keras.layers.BatchNormalization())
    cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_model.add(tf.keras.layers.Dropout(0.25))

    cnn_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    cnn_model.add(tf.keras.layers.BatchNormalization())
    cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_model.add(tf.keras.layers.Dropout(0.25))

    cnn_model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
    cnn_model.add(tf.keras.layers.BatchNormalization())
    cnn_model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    cnn_model.add(tf.keras.layers.Dropout(0.25))

    cnn_model.add(tf.keras.layers.Flatten())
    cnn_model.add(tf.keras.layers.Dense(256, activation='relu'))
    cnn_model.add(tf.keras.layers.BatchNormalization())
    cnn_model.add(tf.keras.layers.Dropout(0.2))
    cnn_model.add(tf.keras.layers.Dense(3, activation='softmax'))
    return cnn_model

In [8]:
print(datetime.datetime.today())
tf.keras.backend.clear_session()
tf.random.set_seed(42)
model=my_cnn_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=.01),
    loss='categorical_crossentropy', 
    metrics=['accuracy'],
)
model.fit(tr_gen, epochs=3, validation_data=val_gen,
      verbose=1)
print(datetime.datetime.today())

2022-12-16 04:47:40.381865
Epoch 1/3


2022-12-16 04:47:55.988209: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2022-12-16 04:47:56.570946: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-12-16 04:47:59.396664: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


     16/Unknown - 243s 15s/step - loss: 2.8497 - accuracy: 0.3500

/tmp/ipykernel_819/1530022409.py:33: RuntimeWarning: invalid value encountered in true_divide
  out=(((out-out.min())/(out.max()-out.min()))*255


1413/1413 [==============================] - 23038s 16s/step - loss: 1.0836 - accuracy: 0.4303 - val_loss: 1.1151 - val_accuracy: 0.4166
Epoch 2/3
1413/1413 [==============================] - 20247s 14s/step - loss: 0.9312 - accuracy: 0.5401 - val_loss: 0.9131 - val_accuracy: 0.5582
Epoch 3/3
1413/1413 [==============================] - 19905s 14s/step - loss: 0.7524 - accuracy: 0.6482 - val_loss: 0.7305 - val_accuracy: 0.6639
2022-12-16 22:20:50.832582


In [9]:
model.save("안진방향판별_v0_1.h5")

## 모델 검증

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2,datetime
from sklearn.metrics import roc_auc_score
print(datetime.datetime.today())

2022-12-18 11:17:12.901218


In [2]:
print(datetime.datetime.today())
model=tf.keras.models.load_model("안진방향판별_v0_1.h5")
print(datetime.datetime.today())

2022-12-18 11:17:12.910961


2022-12-18 11:17:14.255248: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-18 11:17:14.798988: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46723 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:3b:00.0, compute capability: 8.6


2022-12-18 11:17:17.793529


In [3]:
def my_generator(df):
    # 모두 같은 프레임은 제외하고 불러오게 설정
    def avi_to_img(file, video_len=32):
        data=list()
        cap = cv2.VideoCapture(file)
        cnt=0
        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret:
                if (cnt!=0):
                    before=data[-1][0]
                else:
                    before=np.zeros((240,320))

                frame=cv2.resize(frame,(320,240))
                frame=cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
                frame=np.expand_dims(frame/255, axis=0)
                if (before!=frame).any():
                    data.append(frame)
                    cnt+=1
                cv2.waitKey(1)
            else:
                break
            if cnt==video_len:
                break
        cap.release()
        cv2.destroyAllWindows()
        if len(data)!=0:
            out = np.concatenate(data).astype('float')
            # # 이미지별 차이를 리턴
            out=np.diff(out,axis=0)[:].sum(axis=0)
            # # min max scaling 
            out=(((out-out.min())/(out.max()-out.min()))*255
                ).astype('u1')
            out=cv2.cvtColor(out,cv2.COLOR_GRAY2RGB)
            return out.copy()
        else:
            return np.zeros((240,320,3))

    def f():
        for label,video_file in df.sample(frac=1,random_state=42).values:
            video = avi_to_img(video_file)
            y = np.zeros((3))
            if label=="left":
                y[0]=1
            elif label=="right":
                y[2]=1
            else:
                y[1]=1
            yield video, y
    return f

In [4]:
test=pd.read_csv("안진방향판별_test.tsv",sep='\t')
te_dataset=tf.data.Dataset.from_generator(
    my_generator(test),
    output_shapes=((240,320,3), 3),
    output_types=('float32','float32'))
te_gen =te_dataset.batch(640).prefetch(tf.data.experimental.AUTOTUNE)

In [5]:
predict_values=model.predict(te_gen,verbose=1)
print(datetime.datetime.today())

2022-12-18 11:17:58.114472: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2022-12-18 11:17:58.881650: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


      1/Unknown - 35s 35s/step

2022-12-18 11:18:01.787994: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


283/283 [==============================] - 9832s 35s/step
2022-12-18 14:01:19.493127


In [6]:
test['patient_id']=['/'.join(i.split('/')[5:7]) 
                    for i in test.file_path]
test['predict_values']=np.array(['left','mid','right'])[
    np.argmax(predict_values,axis=1)]
test[['patient_id','horizontal','predict_values']].to_csv(
    "안진방향판별_결과값.tsv",index=False,sep='\t')

In [7]:
y_true=np.zeros((test.shape[0],3))
for i,label in enumerate(test.horizontal):
    if label=="left":
        y_true[i,0]=1
    elif label=="right":
        y_true[i,2]=1
    else:
        y_true[i,1]=1

In [8]:
test[['left_true','mid_true','right_true']]=y_true
test[['left_prob','mid_prob','right_prob']]=predict_values

In [9]:
test.loc[:,['patient_id','left_true','mid_true','right_true',
      'left_prob','mid_prob','right_prob']].to_csv(
    '안진방향판별_성능평가계산사용값.tsv',
    index=False,sep='\t')

In [10]:
score={"left":roc_auc_score(y_true[:,0],predict_values[:,0]),
 "mid":roc_auc_score(y_true[:,1],predict_values[:,1]),
 "right":roc_auc_score(y_true[:,2],predict_values[:,2])}
print(score,"total : ",np.mean(list(score.values())))
print(datetime.datetime.today())

{'left': 0.49863576120456415, 'mid': 0.4999792574359456, 'right': 0.5015958283083986} total :  0.5000702823163028
2022-12-18 14:06:45.603700


In [11]:
pd.DataFrame({"AUC":[np.mean(list(score.values()))]}).to_csv(
    "안진방향판별_AUC.tsv",index=False,sep='\t')
print(datetime.datetime.today())

2022-12-18 14:06:54.051393
